# Librerias

In [175]:
import requests
import pandas as pd
from datetime import datetime
import time
import os
import base64

# Extraer

In [176]:
url = 'https://pokeapi.co/api/v2/pokemon?limit=100000&offset=0'
response = requests.get(url)
data = response.json()

In [177]:
save_directory = './pokemon-sprites/'
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

In [178]:
list_pkmn_primary_data = []
list_abilities = []
list_game_appears = []
list_stats = []
list_types = []

for pokemon in data['results']:
    response_info_pokemon = requests.get(pokemon['url'])
    data_info_pokemon = response_info_pokemon.json()
    
    # Datos principales
    id_pokemon = data_info_pokemon['id']
    pokemon_name = pokemon['name']
    pokemon_order = data_info_pokemon['order']
    pokemon_weight = data_info_pokemon['weight']
    pokemon_height = data_info_pokemon['height']
    
    # Datos de habilidades
    pokemon_abilities = data_info_pokemon['abilities']
    for ability in pokemon_abilities:
        id_ability = data_ability_pokemon['id']
        ability_name = ability['ability']['name']
        ability_is_hidden = ability['is_hidden']
        # URL de la habildiad para obtener la descripción
        ability_url = ability['ability']['url']
        response_ability_pokemon = requests.get(ability_url)
        data_ability_pokemon = response_ability_pokemon.json()
        # Obtener la descripcion en ingles de la la habildiad
        for ability in data_ability_pokemon['flavor_text_entries']:
            if ability['language']['name'] == 'en':
                ability_description = ability['flavor_text']
        
        list_abilities.append([id_pokemon, id_ability, ability_name, ability_is_hidden, ability_description])
    
    # Datos de apariciones en versiones
    pokemon_game_indices = data_info_pokemon['game_indices']
    for game in pokemon_game_indices:
        game_name = game['version']['name']
        list_game_appears.append([id_pokemon, game_name])
    # TODO: CONSULTAR API pokemon-species/132 para obtener descripción pokedex validando que sea en ingles

    # Datos de estadistica
    pokemon_stats = data_info_pokemon['stats']
    for stat in pokemon_stats:
        stat_name = stat['stat']['name']
        stat_base = stat['base_stat']
        stat_effort = stat['effort']
        list_stats.append([id_pokemon, stat_name, stat_base, stat_effort])
            
    # Datos de tipo
    pokemon_types = data_info_pokemon['types']
    for type in pokemon_types:
        type_name = type['type']['name']
        type_number = type['slot']
        list_types.append([id_pokemon, type_name, type_number])

    # Descargar el sprite por defecto del pokemon
    front_default_url = data_info_pokemon['sprites']['front_default']
    if front_default_url:
        image_name = f'{id_pokemon}-{pokemon_name}.png'
        image_path = os.path.join(save_directory, image_name)
        img_response = requests.get(front_default_url)

        if img_response.status_code == 200:
            with open(image_path, 'wb') as img_file:
                img_file.write(img_response.content)
                list_pkmn_primary_data.append([id_pokemon, pokemon_order, pokemon_name, pokemon_weight, pokemon_height, image_path])
                print("Registro exitoso: " + pokemon_name)
        else:
            print('Registro exitoso ' + pokemon_name + ' pero falló la descarga del sprite')
    else:
        print('Registro exitoso ' + pokemon_name + ' pero no existe sprite')
        image_path = ''
        list_pkmn_primary_data.append([id_pokemon, pokemon_order, pokemon_name, pokemon_weight, pokemon_height, image_path])
    
    time.sleep(0.1)
    

Registro exitoso: bulbasaur
Registro exitoso: ivysaur
Registro exitoso: venusaur
Registro exitoso: charmander
Registro exitoso: charmeleon
Registro exitoso: charizard
Registro exitoso: squirtle
Registro exitoso: wartortle
Registro exitoso: blastoise
Registro exitoso: caterpie
Registro exitoso: metapod
Registro exitoso: butterfree
Registro exitoso: weedle
Registro exitoso: kakuna
Registro exitoso: beedrill
Registro exitoso: pidgey
Registro exitoso: pidgeotto
Registro exitoso: pidgeot
Registro exitoso: rattata
Registro exitoso: raticate
Registro exitoso: spearow
Registro exitoso: fearow
Registro exitoso: ekans
Registro exitoso: arbok
Registro exitoso: pikachu
Registro exitoso: raichu
Registro exitoso: sandshrew
Registro exitoso: sandslash
Registro exitoso: nidoran-f
Registro exitoso: nidorina
Registro exitoso: nidoqueen
Registro exitoso: nidoran-m
Registro exitoso: nidorino
Registro exitoso: nidoking
Registro exitoso: clefairy
Registro exitoso: clefable
Registro exitoso: vulpix
Registro 

In [179]:
df_pkmn_primary_data_extraer = pd.DataFrame(
    list_pkmn_primary_data,
    columns = ['pkmn_id', 'pokemon_order', 'pokemon_name', 'pokemon_weight', 'pokemon_height', 'pokemon_image_path']
)

df_pkmn_abilities_extraer = pd.DataFrame(
    list_abilities,
    columns = ['pkmn_id', 'ability_id', 'ability_name', 'ability_is_hidden', 'ability_description']
)

df_pkmn_game_appears_extraer = pd.DataFrame(
    list_game_appears,
    columns = ['pkmn_id', 'game_version']
)

df_pkmn_stats_extraer = pd.DataFrame(
    list_stats,
    columns = ['pkmn_id', 'name', 'stat', 'effort']
)

df_pkmn_types_extraer = pd.DataFrame(
    list_types,
    columns = ['pkmn_id', 'name', 'type_number']
)

In [180]:
df_pkmn_primary_data_extraer['extraer_datetime'] = datetime.now()
df_pkmn_abilities_extraer['extraer_datetime'] = datetime.now()
df_pkmn_game_appears_extraer['extraer_datetime'] = datetime.now()
df_pkmn_stats_extraer['extraer_datetime'] = datetime.now()
df_pkmn_types_extraer['extraer_datetime'] = datetime.now()

In [181]:
df_pkmn_primary_data_extraer.to_csv('fuente/extraer/pkmn_primary_data.csv', index = False, sep = '|')
df_pkmn_abilities_extraer.to_csv('fuente/extraer/pkmn_abilities.csv', index = False, sep = '|')
df_pkmn_game_appears_extraer.to_csv('fuente/extraer/pkmn_game_appears.csv', index = False, sep = '|')
df_pkmn_stats_extraer.to_csv('fuente/extraer/pkmn_stats.csv', index = False, sep = '|')
df_pkmn_types_extraer.to_csv('fuente/extraer/pkmn_types.csv', index = False, sep = '|')

# Transformar

In [182]:
df_pkmn_data_transformar = pd.read_csv('fuente/extraer/pkmn_primary_data.csv', delimiter = '|')
df_pkmn_abilities_transformar = pd.read_csv('fuente/extraer/pkmn_abilities.csv', delimiter = '|')
df_pkmn_types_transformar = pd.read_csv('fuente/extraer/pkmn_types.csv', delimiter= '|')
df_pkmn_game_appears_transformar = pd.read_csv('fuente/extraer/pkmn_game_appears.csv', delimiter='|')
df_pkmn_stats_transformar = pd.read_csv('fuente/extraer/pkmn_stats.csv', delimiter='|')

In [183]:
df_pkmn_data_transformar = df_pkmn_data_transformar[['pkmn_id', 'pokemon_order', 'pokemon_name', 'pokemon_weight', 'pokemon_height', 'pokemon_image_path']]
df_pkmn_abilities_transformar = df_pkmn_abilities_transformar[['pkmn_id', 'ability_id', 'ability_name', 'ability_is_hidden', 'ability_description']]
df_pkmn_types_transformar = df_pkmn_types_transformar[['pkmn_id', 'name', 'type_number']]
df_pkmn_game_appears_transformar = df_pkmn_game_appears_transformar[['pkmn_id', 'game_version']]
df_pkmn_stats_transformar = df_pkmn_stats_transformar[['pkmn_id', 'name', 'stat', 'effort']]

In [184]:
# Generar una tabla de habilidades únicas
df_abilities_transformar = df_pkmn_abilities_transformar[['ability_id', 'ability_name', 'ability_description']].drop_duplicates().reset_index(drop=True)

In [185]:
# Crear una tabla de relaciones entre Pokémon y habilidades
df_pkmn_abilities_rel_transformar = df_pkmn_abilities_transformar.copy()
df_pkmn_abilities_rel_transformar = df_pkmn_abilities_rel_transformar[['pkmn_id', 'ability_id', 'ability_is_hidden']]

In [186]:
# Agregar información de la imagen en base64 
save_directory
data_imagenes = []
for image_name in os.listdir(save_directory):
    image_path = os.path.join(save_directory, image_name)

    with open(image_path, 'rb') as img_file:
        base64_string = base64.b64encode(img_file.read()).decode('utf-8')
        # img_html = f'<img src="data:image/png;base64,{base64_string}" width="{300}px" height="{300}px"/>'
        img_html = f'<img src="data:image/png;base64,{base64_string}" style="width:{300}px; height:{300}px;" alt="Image"/>'

    data_imagenes.append({
        'pkmn_id': image_name.split('-')[0],
        'image_name': image_name,
        'image_base64': base64_string,
        'img_html': img_html
    })

df_pkmn_images_transformar = pd.DataFrame(data_imagenes)

In [187]:
df_pkmn_data_transformar['transformar_datetime'] = datetime.now()
df_abilities_transformar['transformar_datetime'] = datetime.now()
df_pkmn_abilities_rel_transformar['transformar_datetime'] = datetime.now()
df_pkmn_types_transformar['transformar_datetime'] = datetime.now()
df_pkmn_game_appears_transformar['transformar_datetime'] = datetime.now()
df_pkmn_stats_transformar['transformar_datetime'] = datetime.now()
df_pkmn_images_transformar['transformar_datetime'] = datetime.now()

In [188]:
df_pkmn_data_transformar.to_csv('fuente/transformar/pkmn_primary_data.csv.csv', sep = '|', index = False)
df_abilities_transformar.to_csv('fuente/transformar/pkmn_abilities.csv.csv', sep = '|', index = False)
df_pkmn_abilities_rel_transformar.to_csv('fuente/transformar/pkmn_abitilies_rel.csv', sep = '|', index = False)
df_pkmn_types_transformar.to_csv('fuente/transformar/pkmn_types.csv', sep = '|', index = False)
df_pkmn_game_appears_transformar.to_csv('fuente/transformar/pkmn_game_appears.csv', sep = '|', index = False)
df_pkmn_stats_transformar.to_csv('fuente/transformar/pkmn_stats.csv', sep = '|', index = False)
df_pkmn_images_transformar.to_csv('fuente/transformar/pkmn_images.csv', sep = '|', index = False)

# Cargar

In [189]:
df_pkmn_data_cargar = pd.read_csv('fuente/transformar/pkmn_primary_data.csv.csv', delimiter = '|')
df_abilities_cargar = pd.read_csv('fuente/transformar/pkmn_abilities.csv.csv', delimiter = '|')
df_pkmn_abilities_rel_cargar = pd.read_csv('fuente/transformar/pkmn_abitilies_rel.csv', delimiter = '|')
df_pkmn_types_cargar = pd.read_csv('fuente/transformar/pkmn_types.csv', delimiter = '|')
df_pkmn_game_appears_cargar = pd.read_csv('fuente/transformar/pkmn_game_appears.csv', delimiter = '|')
df_pkmn_stats_cargar = pd.read_csv('fuente/transformar/pkmn_stats.csv', delimiter = '|')
df_pkmn_images_cargar = pd.read_csv('fuente/transformar/pkmn_images.csv', delimiter = '|')

In [190]:
df_pkmn_data_cargar = df_pkmn_data_cargar[['pkmn_id', 'pokemon_order', 'pokemon_name', 'pokemon_weight', 'pokemon_height', 'pokemon_image_path']]
df_abilities_cargar = df_abilities_cargar[['ability_id', 'ability_name', 'ability_description']]
df_pkmn_abilities_rel_cargar = df_pkmn_abilities_rel_cargar[['pkmn_id', 'ability_id', 'ability_is_hidden']]
df_pkmn_types_cargar = df_pkmn_types_cargar[['pkmn_id', 'name', 'type_number']]
df_pkmn_game_appears_cargar = df_pkmn_game_appears_cargar[['pkmn_id', 'game_version']]
df_pkmn_stats_cargar = df_pkmn_stats_cargar[['pkmn_id', 'name', 'stat', 'effort']]
df_pkmn_images_cargar = df_pkmn_images_cargar[['pkmn_id', 'image_name', 'image_base64', 'img_html']]

In [191]:
df_pkmn_data_cargar['cargar_datetime'] = datetime.now()
df_abilities_cargar['cargar_datetime'] = datetime.now()
df_pkmn_abilities_rel_cargar['cargar_datetime'] = datetime.now()
df_pkmn_types_cargar['cargar_datetime'] = datetime.now()
df_pkmn_game_appears_cargar['cargar_datetime'] = datetime.now()
df_pkmn_stats_cargar['cargar_datetime'] = datetime.now()
df_pkmn_images_cargar['cargar_datetime'] = datetime.now()

In [192]:
df_pkmn_data_cargar.to_csv('fuente/cargar/pkmn_primary_data.csv', sep = '|', index = False)
df_abilities_cargar.to_csv('fuente/cargar/pkmn_abilities.csv', sep = '|', index = False)
df_pkmn_abilities_rel_cargar.to_csv('fuente/cargar/pkmn_abitilies_rel.csv', sep = '|', index = False)
df_pkmn_types_cargar.to_csv('fuente/cargar/pkmn_types.csv', sep = '|', index = False)
df_pkmn_game_appears_cargar.to_csv('fuente/cargar/pkmn_game_appears.csv', sep = '|', index = False)
df_pkmn_stats_cargar.to_csv('fuente/cargar/pkmn_stats.csv', sep = '|', index = False)
df_pkmn_images_cargar.to_csv('fuente/cargar/pkmn_images.csv', sep = '|', index = False)